In [1]:
import json
import re

def transform_json(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)

    for i,item in enumerate(data):
        original_len = len(item.get("conversations", []))
        for conversation in item.get("conversations", []):
            # 原始字符串
            rec_string = conversation.get("rec", "")
            
            # 使用正则表达式匹配标记
            matches = re.finditer(r'<(.*?)>:\s*\[(.*?)\]', rec_string)
            
            # 初始化字典，用于存储标记及其内容
            tag_dict = {}
            
            for match in matches:
                # print(match)
                tag = match.group(1)
                content = match.group(2)
                
                if content.strip():  # 检查内容是否为空
                    # 解析列表内容
                    content_list = [c.strip().strip("'") for c in content.split(",")]
                    tag_dict[f"<{tag}>"] = content_list
            
            # 对可能存在的标签进行检查，确保所有标签都有键值对
            for tag in ["<CAPTION>", "<REASONING>"]:
                if tag not in tag_dict:
                    tag_dict[tag] = []  # 设置为空字符串
            # print(tag_dict)
            # 用新建的字典替代原有的 "rec" 记录
            conversation.update(tag_dict)

            # 移除原始的 "rec" 键
            if "rec" in conversation:
                del conversation["rec"]
        
        
        # 检查“conversations”长度是否一致
        new_len = len(item.get("conversations", []))
        assert original_len == new_len, f"Lengths do not match for item with id: {item.get('id')}. Original: {original_len}, New: {new_len}"
        # if i > 5:
        #     break
    # 将修改后的数据写回JSON文件
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, indent=2)
input_path = "/mnt/pfs-mc0p4k/nlu/team/yuhaofu/model/GCotVLM/GCot_json/24000--.json"
output_path = "/mnt/pfs-mc0p4k/nlu/team/yuhaofu/model/GCotVLM/GCot_json/trans_24000--.json"
# 示例使用
transform_json(input_path, output_path)